In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# Hyper Parameters

lr = 0.001
num_steps = 500
bs = 128
display_step = 50

# Network Parameters
n_in = 784
n_out = 10
pkeep = 0.75 


In [4]:
# Graph Input

X = tf.placeholder("float", [None, n_in])
Y = tf.placeholder("float", [None, n_out])
keep_prob = tf.placeholder("float") 

In [5]:
## Network Variables

weights = {
    # 5X5 convolution filter, 1 input channel(greyscale), 32 outputs channels
    'wc1': tf.Variable(tf.truncated_normal([5,5,1,32], stddev=0.1)),
    
    # 5X5 convolution fileter, 32 inputs channels, 64 outputs channels
    'wc2': tf.Variable(tf.truncated_normal([5,5,32,64], stddev=0.1)),
    
    # After maxpool layer(having stride = 2, and pooling_sz =2, thus reducing size 28*28*64->7*7*64)
    'wd1': tf.Variable(tf.truncated_normal([7*7*64, 1024], stddev=0.1)),
    
    # 1024 inputs -> 10 outputs
    'out': tf.Variable(tf.truncated_normal([1024, n_out], stddev=0.1))
}

biases = {
    # 5X5 convolution filter, 1 input channel(greyscale), 32 outputs channels
    'wc1': tf.Variable(tf.truncated_normal([32], stddev=0.1)),
    
    # After first maxpool layer(having stride = 2, and pooling_sz =2, thus reducing size 28*28*32->14*14*32)
    
    # 5X5 convolution fileter, 32 inputs channels, 64 outputs channels
    'wc2': tf.Variable(tf.truncated_normal([64], stddev=0.1)),
    
    # After last maxpool layer(having stride = 2, and pooling_sz =2, thus reducing size 14*14*64->7*7*64)
    'wd1': tf.Variable(tf.truncated_normal([1024], stddev=0.1)),
    
    # 1024 inputs -> 10 outputs
    'out': tf.Variable(tf.truncated_normal([n_out], stddev=0.1))
}

In [8]:
# Create helper functions

def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1,strides,strides,1], padding='SAME') + b
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1,k,k,1], padding='SAME')


## define model
def conv_net(x, weights, biases, dropout):
    ## mnist data is 784 size vector, reshape it to 28*28
    x = tf.reshape(x, shape=[-1,28,28,1])
    
    # conv layer 1
    conv1 = conv2d(x, weights['wc1'], biases['wc1'])
    # maxpool layer 1
    conv1 = maxpool2d(conv1,k=2)
    
    #conv layer 2
    conv2 = conv2d(conv1, weights['wc2'], biases['wc2'])
    # maxpool layer 2
    conv2 = maxpool2d(conv2, k=2)
    
    
    ## Fully connected layer
    ## Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1,7*7*64])
    fc1 = tf.matmul(fc1, weights['wd1']) + biases['wd1']
    fc1 = tf.nn.relu(fc1)
    
    ## Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)
    
    ## Output layer 1024->10
    out = tf.matmul(fc1, weights['out']) + biases['out']
    return out
    

In [9]:
## Construct model

logits = conv_net(X, weights, biases, keep_prob)
preds = tf.nn.softmax(logits)


## Define loss and optimizer

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss_op)


## Evaluate model
correct_pred = tf.equal(tf.argmax(preds, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

## Initialize the variables
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [ ]:
## Training

with tf.Session() as sess:
    sess.run(init)
    
    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(bs)
        
        sess.run(train_op, feed_dict={X:batch_x, Y:batch_y, keep_prob:pkeep})
        
        if(step%display_step==0):
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X:batch_x, Y:batch_y, keep_prob:pkeep })
            print("Step: ", step, "Minibatch Loss: ", loss, "Minibatch Accuracy: ", acc)
            
    print("training done")
    
    ## Evaluation of our model
    test_acc = sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels, keep_prob:1.0})


Step:  50 Minibatch Loss:  0.28770107 Minibatch Accuracy:  0.890625
Step:  100 Minibatch Loss:  0.2367497 Minibatch Accuracy:  0.90625
Step:  150 Minibatch Loss:  0.061927225 Minibatch Accuracy:  0.9765625
Step:  200 Minibatch Loss:  0.07050916 Minibatch Accuracy:  0.9765625
Step:  250 Minibatch Loss:  0.069633625 Minibatch Accuracy:  0.984375
Step:  300 Minibatch Loss:  0.03143268 Minibatch Accuracy:  0.9921875
Step:  350 Minibatch Loss:  0.029452058 Minibatch Accuracy:  0.984375
Step:  400 Minibatch Loss:  0.014967954 Minibatch Accuracy:  1.0
Step:  450 Minibatch Loss:  0.0130343605 Minibatch Accuracy:  1.0
Step:  500 Minibatch Loss:  0.13267124 Minibatch Accuracy:  0.953125
training done
